In [1]:
import pandas as pd
df = pd.read_csv(r"../data/GridPuzzle.csv")

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274 entries, 0 to 273
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   key          274 non-null    object
 1   id           274 non-null    int64 
 2   question     274 non-null    object
 3   answer       274 non-null    object
 4   Mistral-7b   274 non-null    object
 5   Llama-13b    274 non-null    object
 6   gemini-pro   274 non-null    object
 7   gpt-4-turbo  274 non-null    object
 8   Claude-3     274 non-null    object
dtypes: int64(1), object(8)
memory usage: 19.4+ KB


In [3]:
df.head()

,key,id,question,answer,Mistral-7b,Llama-13b,gemini-pro,gpt-4-turbo,Claude-3
0,341.json,802,Nicholas and Edith are having a small dinner p...,1984 | Annata Branco | gewurztraminer\n1988 | ...,| 1984 | Annata Branco | gewurztraminer |\n |-...,Let's solve the puzzle step by step.\n \n Step...,"Step-by-step solution:\n 1. From clue 6, we kn...","Step-by-step solution:\n \n 1. From clue 6, we...","Step-by-step solution:\n \n 1. From clue 6, we..."
1,341.json,726,The world-famous Punta Santiago Rose Gardening...,first | bon silene | Renee\nsecond | perle d'o...,| first | calocarpa | Hannah\n |---------|----...,Let's break down the clues step by step:\n \n ...,"Step-by-step solution:\n 1. From clue 1, we kn...","Step-by-step solution:\n \n 1. From clue 3, we...","Step-by-step solution:\n \n 1. From clue 1, we..."
2,341.json,16,A standardized test has recently been develope...,55% | Archimedes | CIT\n62% | Galen | UCLA\n69...,| 55% | Zeno | CIT |\n |------|------|-----|\n...,Let's break down the clues and solve the puzzl...,"Step-by-step solution:\n 1. From clue 2, we kn...","Step-by-step solution:\n \n 1. From clue 4, Pr...","Step-by-step solution:\n \n 1. From clue 4, we..."
3,341.json,698,Gil Glabbers has lived on the coast of Guernse...,1962 | Tracy | Cherbourg\n1969 | Felipe | Bour...,| 1962 | Felipe (Bournemouth) |\n | 1969 | Kar...,Let's break down the clues step by step:\n \n ...,"Step-by-step solution:\n \n 1. From clue 2, we...","Step-by-step solution:\n \n 1. From Clue 2, Tr...","Step-by-step solution:\n \n 1. From clue 2, we..."
4,341.json,444,Jamie never misses an opportunity to watch the...,2 | April 22 | Eastbrook\n9 | April 10 | Islet...,| Dates | Locations |\n |---|---|\n | April 10...,Let's break down the clues and solve the puzzl...,"Step-by-step solution:\n \n 1. From clue 1, we...",Step-by-step solution:\n \n Step 1: Analyze th...,"Step-by-step solution:\n \n 1. From clue 3, we..."


In [4]:
from pprint import pprint
pprint(df["question"].iloc[0])

('Nicholas and Edith are having a small dinner party this evening at their '
 "home in Cedar Valley, and they've decided to open a select number of rare "
 'wines from their personal collection to celebrate the occasion. Using only '
 'the clues below, match the vintages to the options from wines and types. '
 'Remember, as with all grid-based logic puzzles, no option in any category '
 'will ever be used more than once.\n'
 '\n'
 'vintages : 1984, 1988, 1992, 1996.\n'
 'wines : Annata Branco, Bianca Flaux, Ece Suss, Vendemmia.\n'
 'types : gewurztraminer, merlot, pinot noir, riesling.\n'
 '\n'
 'Clues:\n'
 '1. The Ece Suss was bottled sometime after the Annata Branco.\n'
 '2. The Bianca Flaux was bottled 4 years before the Vendemmia.\n'
 '3. The 1988 bottle is a pinot noir.\n'
 '4. The merlot is either the Annata Branco or the Bianca Flaux.\n'
 '5. The Bianca Flaux was bottled sometime after the Ece Suss.\n'
 '6. The 1984 bottle is a gewurztraminer.\n'
 '\n'
 'While answering use the 

In [6]:
import re

def num_clues_and_categories(clue, category):
    cat_topic = re.compile(r"[A-za-z]+(?=\s\:\s)")
    topics = cat_topic.findall(category)

    if topics == []:
        topics = None
    
    return len(clue.split("\n")[1:]), len(topics), topics


def get_clues_and_categories(question):
    pattern = re.compile(r"\n\n")
    clues = re.compile(r"Clues:")
    categories = re.compile(r"\w+\s\:\s\w+\s?\w*\,")
    matches = pattern.search(question)

    actual_question = None
    actual_clues = None
    actual_categories = None
    num_clues = 0
    num_categories = 0
    category_names = None

    if matches is not None:
        res = matches.group()
        text_split = question.split(res)
        for ind, val in enumerate(text_split):
            if ind == 0:
                actual_question = val

            if categories.search(val) is not None:
                actual_categories = val
            
            elif clues.search(val) is not None:
                actual_clues = val
    
    if actual_clues is not None and actual_categories is not None:
        num_clues, num_categories, category_names = num_clues_and_categories(actual_clues, actual_categories)


    return actual_question, actual_clues, actual_categories, num_clues, num_categories, category_names


ques = []
clues = []
categories = []
number_clues = []
number_categories = []
category_names= []

for i, row in df.iterrows():
    question = row["question"]
    act_qu, clue, category, num_clues, num_categories, categ = get_clues_and_categories(question)
    
    print(num_clues, num_categories, categ)

    if act_qu is not None and clue is not None and category is not None:
        ques.append(act_qu)
        clues.append(clue)
        categories.append(category)

        if num_clues and num_categories and categ is not None:

            names = " ".join(categ)
            number_clues.append(num_clues)
            number_categories.append(num_categories)
            category_names.append(names)

    else:
        print(i, question)
        print()
        print(clue)
        print()
        print(category)

df["actual_question"] = pd.Series(ques)
df["Clues"] = pd.Series(clues)
df["categories"] = pd.Series(categories)
df["number_of_clues"] = pd.Series(number_clues)
df["number_categories"] = pd.Series(number_categories)
df["category_names"] = pd.Series(category_names)


6 3 ['vintages', 'wines', 'types']
5 3 ['placements', 'roses', 'gardeners']
5 2 ['scores', 'institutions']
5 3 ['sent', 'writers', 'origins']
4 3 ['stars', 'dates', 'locations']
4 3 ['days', 'guests', 'rooms']
5 3 ['lost', 'names', 'diets']
4 3 ['prices', 'winners', 'butterflies']
4 3 ['days', 'names', 'departures']
5 3 ['games', 'players', 'positions']
4 3 ['months', 'owners', 'breeds']
5 3 ['wingspans', 'names', 'ages']
5 3 ['months', 'probes', 'destinations']
4 3 ['days', 'reporters', 'stories']
4 3 ['fares', 'customers', 'points']
5 3 ['dates', 'customers', 'shapes']
5 3 ['years', 'species', 'populations']
5 3 ['decibels', 'sirens', 'years']
5 3 ['distances', 'students', 'colors']
5 3 ['years', 'astronomers', 'comets']
5 3 ['years', 'cultures', 'populations']
4 3 ['years', 'astronomers', 'comets']
5 3 ['years', 'superheros', 'names']
4 3 ['years', 'windmills', 'families']
5 3 ['days', 'companies', 'positions']
5 3 ['orders', 'names', 'items']
5 3 ['years', 'islands', 'cultures']
4 

In [7]:
df.head()

,key,id,question,answer,Mistral-7b,Llama-13b,gemini-pro,gpt-4-turbo,Claude-3,actual_question,Clues,categories,number_of_clues,number_categories,category_names
0,341.json,802,Nicholas and Edith are having a small dinner p...,1984 | Annata Branco | gewurztraminer\n1988 | ...,| 1984 | Annata Branco | gewurztraminer |\n |-...,Let's solve the puzzle step by step.\n \n Step...,"Step-by-step solution:\n 1. From clue 6, we kn...","Step-by-step solution:\n \n 1. From clue 6, we...","Step-by-step solution:\n \n 1. From clue 6, we...",Nicholas and Edith are having a small dinner p...,Clues:\n1. The Ece Suss was bottled sometime a...,"vintages : 1984, 1988, 1992, 1996.\nwines : An...",6,3,vintages wines types
1,341.json,726,The world-famous Punta Santiago Rose Gardening...,first | bon silene | Renee\nsecond | perle d'o...,| first | calocarpa | Hannah\n |---------|----...,Let's break down the clues step by step:\n \n ...,"Step-by-step solution:\n 1. From clue 1, we kn...","Step-by-step solution:\n \n 1. From clue 3, we...","Step-by-step solution:\n \n 1. From clue 1, we...",The world-famous Punta Santiago Rose Gardening...,Clues:\n1. The rose that won fourth place was ...,"placements : first, second, third, fourth.\nro...",5,3,placements roses gardeners
2,341.json,16,A standardized test has recently been develope...,55% | Archimedes | CIT\n62% | Galen | UCLA\n69...,| 55% | Zeno | CIT |\n |------|------|-----|\n...,Let's break down the clues and solve the puzzl...,"Step-by-step solution:\n 1. From clue 2, we kn...","Step-by-step solution:\n \n 1. From clue 4, Pr...","Step-by-step solution:\n \n 1. From clue 4, we...",A standardized test has recently been develope...,Clues:\n1. Zeno finished with a score that was...,"scores : 55%, 62%, 69%, 76%.\na.i. : Archimede...",5,2,scores institutions
3,341.json,698,Gil Glabbers has lived on the coast of Guernse...,1962 | Tracy | Cherbourg\n1969 | Felipe | Bour...,| 1962 | Felipe (Bournemouth) |\n | 1969 | Kar...,Let's break down the clues step by step:\n \n ...,"Step-by-step solution:\n \n 1. From clue 2, we...","Step-by-step solution:\n \n 1. From Clue 2, Tr...","Step-by-step solution:\n \n 1. From clue 2, we...",Gil Glabbers has lived on the coast of Guernse...,Clues:\n1. The message from Plymouth was sent ...,"year sent : 1962, 1969, 1976, 1983.\nwriters :...",5,3,sent writers origins
4,341.json,444,Jamie never misses an opportunity to watch the...,2 | April 22 | Eastbrook\n9 | April 10 | Islet...,| Dates | Locations |\n |---|---|\n | April 10...,Let's break down the clues and solve the puzzl...,"Step-by-step solution:\n \n 1. From clue 1, we...",Step-by-step solution:\n \n Step 1: Analyze th...,"Step-by-step solution:\n \n 1. From clue 3, we...",Jamie never misses an opportunity to watch the...,Clues:\n1. The outing to Eastbrook logged 7 fe...,"shooting stars : 2, 9, 16, 23.\ndates : April ...",4,3,stars dates locations


In [8]:
df.to_csv(r"../data/GridPuzzle_processed.csv", index=False)